## Problem 2: Simple Multinomial Processing Trees

In [39]:
import stan
import arviz as az
import nest_asyncio
nest_asyncio.apply()

In [40]:
# Recognition memory task data
data = {
    "hits": 75,
    "misses": 25,
    "false_alarms": 30,
    "correct_rejects": 70
}

### The One-High-Threshold Model (1HT)

In [41]:
# Define the 1HT Stan model
model_1ht_code = """
data {
  int<lower=0> hits;
  int<lower=0> misses;
  int<lower=0> false_alarms;
  int<lower=0> correct_rejects;
}

parameters {
  real<lower=0, upper=1> D;
  real<lower=0, upper=1> g;
}

model {
  hits ~ binomial(hits + misses, D + (1 - D) * g);
  false_alarms ~ binomial(false_alarms + correct_rejects, g);
}

generated quantities {
  vector[2] log_lik;
  log_lik[1] = binomial_lpmf(hits | hits + misses, D + (1 - D) * g);
  log_lik[2] = binomial_lpmf(false_alarms | false_alarms + correct_rejects, g);
}
"""

In [42]:
# Build and sample from 1HT model
posterior_1ht = stan.build(model_1ht_code, data=data)
fit_1ht = posterior_1ht.sample(num_chains=4, num_samples=1000)

# Convert to ArviZ format
idata_1ht = az.from_pystan(posterior=fit_1ht, log_likelihood="log_lik")

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%


Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.29 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!


In [43]:
az.summary(idata_1ht, var_names=["D", "g"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
D,0.629,0.067,0.502,0.751,0.002,0.001,1724.0,2043.0,1.0
g,0.308,0.046,0.227,0.398,0.001,0.001,1941.0,2301.0,1.0


### The Two-High-Threshold Model (2HT)

In [44]:
# Define the 2HT Stan model as a string
model_2ht_code = """
data {
  int<lower=0> hits;
  int<lower=0> misses;
  int<lower=0> false_alarms;
  int<lower=0> correct_rejects;
}

parameters {
  real<lower=0, upper=1> Do;
  real<lower=0, upper=1> Dn;
  real<lower=0, upper=1> g;
}

model {
  hits ~ binomial(hits + misses, Do + (1 - Do) * g);
  false_alarms ~ binomial(false_alarms + correct_rejects, (1 - Dn) * g);
}

generated quantities {
  vector[2] log_lik;
  log_lik[1] = binomial_lpmf(hits | hits + misses, Do + (1 - Do) * g);
  log_lik[2] = binomial_lpmf(false_alarms | false_alarms + correct_rejects, (1 - Dn) * g);
}
"""

In [45]:
# Build and sample from 2HT model
posterior_2ht = stan.build(model_2ht_code, data=data)
fit_2ht = posterior_2ht.sample(num_chains=4, num_samples=1000)

# Convert to ArviZ format
idata_2ht = az.from_pystan(posterior=fit_2ht, log_likelihood="log_lik")

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%


Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!


In [46]:
az.summary(idata_2ht, var_names=["Do", "Dn", "g"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Do,0.420,0.189,0.049,0.703,0.007,0.005,726.0,723.0,1.0
Dn,0.392,0.179,0.022,0.639,0.007,0.005,780.0,831.0,1.0
g,0.529,0.144,0.288,0.767,0.005,0.004,750.0,1738.0,1.0


### LOO Model Comparison

In [47]:
az.compare({"1HT": idata_1ht, "2HT": idata_2ht})

/opt/anaconda3/lib/python3.12/site-packages/arviz/stats/stats.py:795: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/arviz/stats/stats.py:795: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
2HT,0,-6.889052,1.353385,0.000000,1.0,0.05492,0.00000,True,log
1HT,1,-7.428831,1.893953,0.539779,0.0,0.37396,0.31904,True,log


**Posterior Estimates:**

1HT Model:

- D (detection of old items): mean = 0.63, narrow credible interval → suggests good ability to recognize old items.

- g (guessing rate): mean = 0.31, fairly low → guessing not dominant in decisions.

2HT Model:

- Do (detect old): mean = 0.42, more uncertain (wider interval)

- Dn (detect new): mean = 0.39, also uncertain

- g: mean = 0.53, noticeably higher than in 1HT → indicating greater reliance on guessing when detection fails

This suggests the 2HT model attributes more behavior to guessing and less to reliable detection.

**Model Fit (LOO Comparison):**

2HT model ranks better (lower LOO error), indicating it fits the data better. However, the difference in ELPD (~0.54) is small, and standard errors (se, dse) are relatively large compared to the difference → model improvement is not strongly conclusive.

**Conclusion:**

The 2HT model slightly outperforms the 1HT model in explaining the recognition memory data. It allows for detection of both old and new items, and estimates a higher guessing rate, suggesting participants might rely more on guessing when detection fails. The improvement is modest and uncertain, so further data or individual-level modeling could help confirm which model is more appropriate.